# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import sys
sys.path.append('../../../')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,str,f64,str,str,bool
"""../../../../da…","""38531247799057…","""74514634382439…",1.2927e19,"""11276312873930…","""15824663000456…",false
"""../../../../da…","""50060751709436…","""11516187759915…",1.4856e19,"""11276312873930…","""69232738458648…",true
"""../../../../da…","""10179697731576…","""10206684803586…",5.6458e17,"""12983908670116…","""15824663000456…",false
"""../../../../da…","""50060751709436…","""12844266501975…",4.0723e18,"""74726269259809…","""15824663000456…",false
"""../../../../da…","""38531247799057…","""95740663638628…",1.4677e19,"""74726269259809…","""39621191195074…",true
"""../../../../da…","""10179697731576…","""15508239920557…",1.6330e19,"""40644369389378…",null,false
"""../../../../da…","""10179697731576…","""18403526683461…",8.7675e18,"""74726269259809…","""39621191195074…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'Customer_City': 'PREDICTOR_2',
 'Customer_CLV': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder='../../../../data/',
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""../../../../da…","""98189861281952…","""15146581906312…",1.2927e19,"""FirstMortgage3…","""11173092033416…","""Rejected"""
"""../../../../da…","""54408371297395…","""11283163751089…",1.4856e19,"""FirstMortgage3…","""61909326753733…","""Accepted"""
"""../../../../da…","""12417782891543…","""71724905022193…",5.6458e17,"""MoneyMarketSav…","""11173092033416…","""Rejected"""
"""../../../../da…","""54408371297395…","""10271379074945…",4.0723e18,"""BasicChecking""","""11173092033416…","""Rejected"""
"""../../../../da…","""98189861281952…","""61116809072641…",1.4677e19,"""BasicChecking""","""12873539048473…","""Accepted"""
"""../../../../da…","""12417782891543…","""43672394736898…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""12417782891543…","""12351434122536…",8.7675e18,"""BasicChecking""","""12873539048473…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""../../../../da…","""12201179972037…","""32776048454305…",1.2927e19,"""FirstMortgage3…","""67466781592698…","""Rejected"""
"""../../../../da…","""10387954017491…","""17542996113574…",1.4856e19,"""FirstMortgage3…","""12516058878803…","""Accepted"""
"""../../../../da…","""68895133942764…","""76254892061477…",5.6458e17,"""MoneyMarketSav…","""67466781592698…","""Rejected"""
"""../../../../da…","""10387954017491…","""15693247401202…",4.0723e18,"""BasicChecking""","""67466781592698…","""Rejected"""
"""../../../../da…","""12201179972037…","""13297783642379…",1.4677e19,"""BasicChecking""","""10432716814319…","""Accepted"""
"""../../../../da…","""68895133942764…","""16485367939758…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""68895133942764…","""62821819395448…",8.7675e18,"""BasicChecking""","""10432716814319…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_MaritalStatus=PREDICTOR_1
Customer_City=PREDICTOR_2
Customer_CLV=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""16736394067969…","""18282423764750…",1.2927e19,"""FirstMortgage3…","""18272059495901…","""Rejected"""
"""13800566454611…","""10699049831208…",1.4856e19,"""FirstMortgage3…","""13014437798982…","""Accepted"""
"""13766911688903…","""41887238179429…",5.6458e17,"""MoneyMarketSav…","""18272059495901…","""Rejected"""
"""13800566454611…","""12220168637682…",4.0723e18,"""BasicChecking""","""18272059495901…","""Rejected"""
"""16736394067969…","""15766058094250…",1.4677e19,"""BasicChecking""","""17990249299604…","""Accepted"""
"""13766911688903…","""11972766348848…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""13766911688903…","""69056436566025…",8.7675e18,"""BasicChecking""","""17990249299604…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'